In [3]:
import pandas as pd

# data loading / analysis

In [4]:
# load 'Alternative EV Screen.xlsx'  
alt_ev_screen_df = pd.read_excel('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/Alternative EV Screen.xlsx', sheet_name='Cleaned')


print(len(alt_ev_screen_df))
print(alt_ev_screen_df.columns) 
display(alt_ev_screen_df.head())

52
Index(['Ticker', 'Last Price', 'YTD Returns', 'Cover Days', 'SI', 'NT Uptrend',
       'Intermediate Term Uptrend', '20 DMA', '50 DMA', '100 DMA', '200 DMA',
       '2Q24', '3Q24', '4Q24', 'ADV $M', 'Cash Runway (Qs)', 'Market Cap',
       'Total Debt', 'Current', '2Q24 EV', '3Q24 EV', 'Cash on Hand',
       'Net Loss (Qs)', '20 DMA.1', '50 DMA.1', '100 DMA.1', '200 DMA.1'],
      dtype='object')


/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Ticker,Last Price,YTD Returns,Cover Days,SI,NT Uptrend,Intermediate Term Uptrend,20 DMA,50 DMA,100 DMA,...,Total Debt,Current,2Q24 EV,3Q24 EV,Cash on Hand,Net Loss (Qs),20 DMA.1,50 DMA.1,100 DMA.1,200 DMA.1
0,PRE,4.7700,-0.192893,1.056601,0.003922,-0.010658,-0.074336,0.035944,0.024903,-0.089601,...,2.369388,-124.177628,-112.677628,-101.177628,183.873483,-11.500,4.604495,4.654098,5.239461,5.027851
1,ANTX,1.1000,-0.946315,1.365465,0.006300,-0.007799,-0.729220,0.041667,0.033543,-0.334483,...,0.000000,-71.713326,-60.213326,-48.713326,104.540000,-11.500,1.056,1.064300,1.652850,3.930500
2,BOLT,0.6596,-0.411039,1.523604,0.003992,-0.000992,-0.282287,0.019971,0.018959,-0.044480,...,20.219000,-56.729132,-43.277132,-29.825132,102.189000,-13.452,0.646685,0.647327,0.690304,0.901930
3,ALGS,8.7500,-0.472892,1.754932,0.033707,-0.262578,-0.327272,0.128086,-0.168124,-0.230097,...,11.138000,-55.432097,-46.087097,-36.742097,94.536000,-9.345,7.7565,10.518400,11.365075,15.635444
4,KZR,0.7651,-0.192421,0.671468,0.017282,0.191504,-0.093192,-0.061710,0.117976,0.164678,...,18.933000,-89.425605,-66.425605,-43.425605,164.182000,-23.000,0.81542,0.684362,0.656920,0.754693


# data mining / validation

In [5]:
import requests
from datetime import datetime
import time

In [10]:
def get_company_info(cik, headers):
    """
    Get company SIC code and industry info from SEC
    """
    try:
        cik_padded = str(cik).zfill(10)
        url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'
        
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return {
                'sic': data.get('sicCode', ''),
                'industry': data.get('sicDescription', ''),
                'ticker': data.get('tickers', [''])[0] if data.get('tickers') else ''
            }
    except Exception as e:
        print(f"Error fetching company info for CIK {cik}: {str(e)}")
    return None

def get_delistings(start_year=2020, save_csv=True):
    """
    Get Form 25 delistings using SEC's HTTPS access
    Returns: DataFrame of delistings
    Optional: Saves CSV file if save_csv=True
    """
    headers = {
        'User-Agent': 'SPS matt@sanspeursystems.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'www.sec.gov'
    }

    all_delistings = []
    current_year = datetime.now().year

    try:
        for year in range(start_year, current_year + 1):
            for quarter in range(1, 5):
                url = f'https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{quarter}/master.idx'
                
                try:
                    print(f"Fetching {year} Q{quarter}...")
                    response = requests.get(url, headers=headers)
                    
                    if response.status_code == 200:
                        lines = response.text.split('\n')
                        data_started = False
                        
                        for line in lines:
                            if data_started:
                                if '|25|' in line:
                                    parts = line.split('|')
                                    if len(parts) >= 4:
                                        cik = parts[0]
                                        # company_info = get_company_info(cik, headers)
                                        
                                        delisting = {
                                            'date': parts[3],
                                            'company': parts[1],
                                            'cik': cik,
                                            'form': '25'
                                            # 'sic': company_info.get('sic', ''),
                                        }
                                        all_delistings.append(delisting)
                            elif line.startswith('---'):
                                data_started = True
                                
                    time.sleep(0.1)  # SEC rate limiting
                    
                except requests.exceptions.RequestException as e:
                    print(f"Error fetching {year} Q{quarter}: {str(e)}")
                    continue

        if all_delistings:
            df = pd.DataFrame(all_delistings)
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date', ascending=False)
            
            # Save CSV if requested
            if save_csv:
                csv_filename = f'all_delistings_since_{start_year}.csv'
                df.to_csv(csv_filename, index=False)
                print(f"\nSaved to {csv_filename}")
            
            return df
        else:
            print("No delistings found")
            return None

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

In [ ]:
# create function to run get_delistings
def get_delistings_since(year, save_csv=False):
    print(f"Fetching delistings since {year}...")
    return get_delistings(start_year=year, save_csv=save_csv)

# run get_delistings_since
dl_since_df = get_delistings_since(2016)

In [13]:
print(len(dl_since_df))
# display(dl_since_df.head())
display(dl_since_df.tail())

,date,company,cik,form
15,2016-01-27,iSHARES INC,930667,25
1,2016-01-27,iSHARES TRUST,1100663,25
11,2016-01-25,TASMAN METALS LTD.,1474547,25
2,2016-01-21,MARKET VECTORS ETF TRUST,1137360,25
3,2016-01-04,WILLIS GROUP HOLDINGS PLC,1140536,25


enriching the data for delisted stocks from {x} year.

In [ ]:
# clean up the CIK numers
dl_since_df['cik'] = dl_since_df['cik'].astype(str).str.zfill(10)
# convert 'date' string to datetime
dl_since_df['date'] = pd.to_datetime(dl_since_df['date'])

# add empty columns for SEC data 
dl_since_df['ticker'] = '' 
dl_since_df['sic'] = ''
dl_since_df['sic_description'] = ''
dl_since_df['is_biotech'] = False
dl_since_df['biotech_identification_method'] = ''

In [ ]:
# define biotech-related keywords and patterns, comprehensive list
biotech_keywords = [
    'therapeutics', 'pharma', 'biotech', 'bio', 'health sciences',
    'medical', 'genetics', 'genomics', 'life sciences', 'biologics',
    'biosciences', 'oncology', 'diagnostic', 'biomedical',
    'laboratory', 'clinical', 'research', 'drug', 'therapeutic',
    'cell therapy', 'gene therapy', 'molecular', 'pharmaceutical'
]

# biotech and related SIC codes with descriptions comprehensive dictionary
biotech_sic_dict = {
    '2833': 'Medicinal Chemicals and Botanical Products',
    '2834': 'Pharmaceutical Preparations',
    '2835': 'In Vitro and In Vivo Diagnostic Substances',
    '2836': 'Biological Products, Except Diagnostic Substances',
    '3826': 'Laboratory Analytical Instruments',
    '3841': 'Surgical and Medical Instruments',
    '3842': 'Orthopedic, Prosthetic, and Surgical Supplies',
    '3843': 'Dental Equipment and Supplies',
    '3844': 'X-Ray Apparatus and Tubes',
    '3845': 'Electromedical and Electrotherapeutic Apparatus',
    '3851': 'Ophthalmic Goods',
    '8071': 'Medical Laboratories',
    '8731': 'Commercial Physical and Biological Research',
    '8732': 'Commercial Nonphysical Research',
    '8733': 'Noncommercial Research Organizations',
    '8734': 'Testing Laboratories',
    '8999': 'Services, Not Elsewhere Classified (including Medical Research)',
    '6324': 'Hospital and Medical Service Plans',
    '5122': 'Drugs, Proprietaries, and Druggists Sundries'
}
# create object for biotech SIC codes
biotech_sic_codes = list(biotech_sic_dict.keys())

In [ ]:
# SEC API request headers
headers = {
    'User-Agent': 'sanspeursystems matt@sanspeursystems.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

In [ ]:
# Process each company
for idx, row in dl_since_df.iterrows():
    print(f"Processing {row['company']} ({idx + 1}/{len(dl_since_df)})")
    
    # Check company name for biotech keywords
    company_name_lower = row['company'].lower()
    matching_keywords = [kw for kw in biotech_keywords if kw in company_name_lower]
    if matching_keywords:
        dl_since_df.at[idx, 'is_biotech'] = True
        dl_since_df.at[idx, 'biotech_identification_method'] = f"name_match:{','.join(matching_keywords)}"
    
    # Get SEC data
    cik = row['cik']
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            # Store basic info
            dl_since_df.at[idx, 'ticker'] = data.get('tickers', [''])[0]
            sic_code = str(data.get('sic', '')).strip()
            dl_since_df.at[idx, 'sic'] = sic_code
            dl_since_df.at[idx, 'sic_description'] = data.get('sicDescription', '') or biotech_sic_dict.get(sic_code, '')
            
            # Check if SIC code matches biotech
            if sic_code in biotech_sic_codes:
                dl_since_df.at[idx, 'is_biotech'] = True
                current_method = dl_since_df.at[idx, 'biotech_identification_method']
                if current_method:
                    dl_since_df.at[idx, 'biotech_identification_method'] = f"{current_method};sic_{sic_code}"
                else:
                    dl_since_df.at[idx, 'biotech_identification_method'] = f"sic_{sic_code}"
            
            print(f"Company: {row['company']}")
            print(f"SIC: {sic_code} - {dl_since_df.at[idx, 'sic_description']}")
            print(f"Ticker: {dl_since_df.at[idx, 'ticker']}")
            print(f"Is Biotech: {dl_since_df.at[idx, 'is_biotech']}")
            print("---")
            
    except Exception as e:
        print(f"Error for {row['company']}: {str(e)}")
    
    time.sleep(0.1)  # Respect SEC rate limits
